In [1]:

def get_request(q):
    return(spotify.search(q, limit=1, offset=0, type='track', market="GB")['tracks']['items'][0])
    
def get_track_info(artistName, trackName):
    original_artistName = artistName
    original_trackName = trackName
    i = 1
    while i<=3:
        try:
            q = "artist:" + artistName + " track:" + trackName
            data = get_request(q)
            break
        except:
            if i == 1:
                artistName = re.sub("(\[.*\])|(\(.*\))|(\*.*\*)",  '',    artistName).strip()  
                trackName = re.sub("(\[.*\])|(\(.*\))|(\*.*\*)",  '',    trackName).strip()
            if i == 2:
                artistName = re.sub("[',’]",  '',    artistName).strip()  
                trackName = re.sub("[',’]",  '',    trackName).strip()
            i += 1
            
    if i == 4:
        print('Error with: ', artistName, ":", trackName)
        return(pd.DataFrame(),pd.DataFrame(),pd.DataFrame(),pd.DataFrame())
    
    return(clean_track_data(data, 
                            search = True, 
                            original_artist_name = original_artistName,
                            original_track_name = original_trackName,
                            artist_name = artistName,
                            track_name = trackName,
                            q_name = q))

def clean_track_data(data, 
                     search = False, 
                     original_artist_name = None,
                     original_track_name = None,
                     artist_name = None,
                     track_name = None,
                     q_name = None):
    artist_data = data['artists']
    album_data = data['album']
    album_artists = album_data['artists']
    data.pop('album',None)
    data.pop('artists',None)

    artist_data_df = pd.concat([pd.DataFrame(artist_data),pd.DataFrame(album_artists)])
    artist_data_df.rename(columns = {'external_urls': "Artist External URL's",
                        'href': "Artist href",
                        'id': "Artist ID",
                        'name': "Artist Name",
                        'type': "Artist Type",
                        'uri': "Artist URI"}, inplace=True)
    artist_data_df['Artist ID'] = artist_data_df.apply(lambda x: x['Artist ID'] if not pd.isna(x['Artist ID']) else 'local' + hash_(x['Artist Name']), axis = 1)
    artist_data_df = artist_data_df.drop_duplicates(subset='Artist ID', keep='first')

    album_data['artists'] = [album_artist['id'] for album_artist in album_data['artists']]
    album_data_df = pd.DataFrame({k: [v] for k, v in album_data.items()})
    album_data_df.rename(columns={'album_type': "Album Type", 
                            'artists': "Album Artists", 
                            'external_urls': "External URL's", 
                            'href': "href", 
                            'id': "Album ID", 
                            'images': "Album Images",
                            'name': "Album Name", 
                            'release_date': "Album Release Date", 
                            'release_date_precision': "Album Release Date Precision", 
                            'total_tracks': "Album Total Tracks",
                            'type': "Album Object Type", 
                            'uri': "Album URI"}, inplace=True)
    #album_data_df['Album ID'] = album_data_df.apply(lambda x: x['Album ID'] if not pd.isna(x['Album ID']) else 'local' + hash_(x['Album Name']), axis = 1)
    album_data_df.drop(labels='available_markets', axis=1, inplace=True, errors='ignore')
    album_data_df = album_data_df[~album_data_df['Album ID'].apply(lambda x: pd.isna(x))]
    
    data['Album ID'] = album_data['id']
    data["Artist ID's"] = tuple(artist_data_df['Artist ID']) 
    #data["Artist ID's"] = tuple(artist['id'] for artist in artist_data)
    track_data_df = pd.DataFrame({k: [v] for k, v in data.items()})
    track_data_df.rename(columns={'disc_number': 'Disc Number', 
                            'duration_ms': 'Track Duration (ms)', 
                            'explicit': 'Is Track Explicit', 
                            'external_ids': "Track External ID's",
                            'external_urls': "Track External URL's",
                            'href': 'Track href', 
                            'id': 'Track ID', 
                            'is_local': 'Is Track Local', 
                            'is_playable': 'Is Track Playable', 
                            'name': 'Track Name',
                            'popularity': 'Track Popularity', 
                            'preview_url': 'Track Preview URL', 
                            'track_number': 'Track Number', 
                            'type': 'Track Type', 
                            'uri': 'Track URI'}, inplace=True)
    track_data_df.loc[track_data_df['Is Track Local'] == True,'Track ID'] = ['local' + hash_(x) for x in track_data_df.loc[track_data_df['Is Track Local'] == True,'Track Name']]
    #tracks['Track ID'] = tracks.apply(lambda x: x['Track ID'] if x['Is Track Local'] == False else 'local' + hash_(x['Track Name']), axis = 1)
    track_data_df.drop(labels=['available_markets','episode','track'], axis=1, inplace=True, errors='ignore')
    
    search_data_df = pd.DataFrame()
    if search == True:
        search_data_df['Original Search Artist'] = [original_artist_name]
        search_data_df['Original Search Track'] = [original_track_name]
        search_data_df['Cleaned Search Artist'] = [artist_name]
        search_data_df['Cleaned Search Track'] = [track_name]
        search_data_df['Search Query'] = [q_name]
        search_data_df['Track ID'] = track_data_df['Track ID']
    
    return(track_data_df,album_data_df,artist_data_df,search_data_df)